## 0. 데이터 불러오기

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

products = pd.read_csv("dataset/products.csv")
users = pd.read_csv("dataset/users.csv")
ratings = pd.read_csv("dataset/ratings.csv")

In [ ]:
print(f"products: {products.shape}")
display(products.head())

print(f"users: {users.shape}")
display(users.head())

print(f"ratings: {ratings.shape}")
display(ratings.head())

## 1. 내용 기반 추천 (상품 설명 + 태그 + 카테고리 활용)
- 속성 기반 매칭 → 동일 속성을 공유하는 아이템을 우선 추천(카테고리/태그 기반 필터링)
- 머신러닝 기반 → 아이템의 속성을 독립 변수로, 사용자가 좋아할 확률을 추정
- 속성 기반 유사도 → 아이템 관련 속성들 임베딩 후 유사도 측정 방법론 사용

### 1) 데이터 전처리

In [ ]:
# 텍스트 특성 결합
# 휴대용, 터치스크린, 최신 기술과 소음 제거 기능 → 휴대용 터치스크린 최신 기술과 소음 제거 기능

products["features"] = products["description"] + " " + products["tags"].str.replace(",", "") + " " + products["category"]

products.head()

### 2) 데이터 벡터화

#### [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

- Bag of Words(단어들의 출현 빈도에만 집중하는 텍스트 데이터 수치화 표현 방법) 기법을 활용한 벡터화 방식
- 문서 내에서 단어의 등장 횟수를 기반으로 벡터를 생성
- 빈도만을 고려하기 때문에 모든 단어가 같은 중요도를 가짐
---
- 예시
  - 문서: `"yesterday all my trouble seem so far away"`
  - 단어 사전: `[yesterday, all, my, trouble, seem, far]`
  - 벡터: `[1, 1, 1, 1, 1, 1]`(각 단어가 한 번씩 등장했으므로 모두 1)

---

#### [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

- TF-IDF(Term Frequency-Inverse Document Frequency) 기법을 기반으로 벡터 변환을 수행
- 단어의 빈도와 역 문서 빈도를 사용하여 단어들마다 중요한 정도에 따라서 가중치를 부여하는 방법
- TF (Term Frequency, 단어 빈도) : 한 문서 안에서 어떤 단어가 얼마나 자주 나오는지를 보는 값
- IDF (Inverse Document Frequency, 역문서 빈도) : 흔한 단어일수록 덜 중요하게, 드문 단어일수록 더 중요하게 보정하는 값
- 문서에 많이 등장하는 단어는 중요하다고 가정하지만, 모든 문서에 걸쳐 자주 등장하는 단어(the, is, at, this 등)는 정보량이 적기 때문에 가중치가 낮아진다.
---
- 예시
  - 문서: `"yesterday all my trouble seem so far away"`
  - 단어 사전: `[yesterday, all, my, trouble, seem, far]`
  - 벡터: `[0.52, 0.30, 0.30, 0.45, 0.45, 0.40]`(`yesterday`, `trouble` 같은 특정성이 높은 단어에 더 큰 가중치가 부여됨)

In [ ]:
# TfidfVectorizer 적용


### 3) 유사도 계산 - 코사인 유사도

| user_id |  |  |  |  |
|----------|-----|-----|-----|-----|
| 1        | 5.0 | 4.0 | 3.0 | 0.0 |
| 2        | 4.0 | 0.0 | 0.0 | 5.0 |
| 3        | 0.0 | 2.0 | 3.0 | 0.0 |
| 4        | 5.0 | 0.0 | 4.0 | 2.0 |
---
#### Step 1: 두 사용자의 평점을 벡터로 표현
사용자 1과 사용자 2의 평점 벡터를 추출합니다.

- 사용자 1의 벡터 : A = [5.0, 4.0, 3.0, 0.0]
- 사용자 2의 벡터 : B = [4.0, 0.0, 0.0, 5.0]
---
#### Step 2: 벡터 내적 계산
각 원소 간의 곱을 더한다.
- `A⋅B=(5.0×4.0)+(4.0×0.0)+(3.0×0.0)+(0.0×5.0)=20.0+0.0+0.0+0.0=20.0`
---
#### Step 3: 벡터의 크기(유클리드 거리) 계산

- A_Euclid : `np.sqrt(25 + 14 + 9)`
- B_Euclid : `np.sqrt(16 + 25)`
---
#### Step 4: 코사인 유사도 계산

- Cosine Similarity = `A⋅B / (A_Euclid + B_Euclid)`
---
#### Step 5: 나머지 사용자들에 대해서도 반복
- user1 - user1 -> 1
- user1 - user2 -> 0.442
- user1 - user3 -> 0.667
- user1 - user4 -> 0.779
- 최종적으로 user1에 대한 코사인 유사도 결과 행 생성 -> [1, 0.442, 0.667, 0.779]

In [ ]:
# 1) 코사인 유사도 계산


In [ ]:
# 2) 유사도 점수 정렬


In [ ]:
# 3) 추천할 상품 인덱스 출력 (자기 자신 제외)


### 4) 추천 함수 생성

In [ ]:
def contents_recommendation(dataframe, sim_matrix, product_id, top_k=7):
    print("🔹 콘텐츠 기반 추천")

## 2. 사용자 기반 협업 필터링
- 사용자들이 아이템들을 어떻게 평가했는지를 분석하여 비슷하게 분석한 사용자들이 선호하는 아이템을 추천

### 1) USER-ITEM 행렬 생성

In [ ]:
user_item_matrix = ratings.pivot_table(
    index="user_id",
    columns="product_id",
    values="rating",
    aggfunc="mean",
    fill_value=0
    )
    
user_item_matrix

### 2) 유사도 계산

In [ ]:
# 1) 코사인 유사도(사용자-사용자) 계산


In [ ]:
# 2) 유사도 점수 정렬


In [ ]:
# 3) 사용자가 구매하지 않은 추천 아이템 ID 추출


In [ ]:
# 4) 상품 테이블에서 추천 싱품 출력


### 3) 추천 함수 생성

In [ ]:

def user_col_filtering(dataframe, sim_matrix, user_id, top_k=7):

    print("🔹 사용자 기반 협업 필터링")

## 3. 아이템 기반 협업 필터링
- 사용자들이 아이템들을 어떻게 평가했는지를 분석하여 비슷한 흐름으로 평가받은 아이템을 추천

### 1) 유사도 계산

In [ ]:
# 1) 코사인 유사도(아이템-아이템) 계산


In [ ]:
# 2) 유사도 점수 정렬


In [ ]:
# 3) 유사도가 높은 아이템 ID 추출


In [ ]:
# 4) 상품 테이블에서 추천 싱품 출력


### 2) 추천 함수 생성

In [ ]:
def item_col_filtering(dataframe, sim_matrix, product_id, top_k=7):
    print("🔹 아이템 기반 협업 필터링")

## 4. 하이브리드 추천 (콘텐츠 기반 + 협업 필터링 결합)
- 내용 기반 추천 + 유저 기반 협업필터링 + 아이템 기반 협업필터링
- 세 가지 방법들에 각각 일정 가중치를 부여하여 최종 상품을 추천

In [ ]:
target_item = 1
target_user = 10

content_rec = contents_recommendation(
    dataframe=products,
    sim_matrix=cosine_sim,
    product_id=target_item,
    )

user_rec = user_col_filtering(
    dataframe=products,
    sim_matrix=user_sim,
    user_id=target_user,
    )

item_rec = item_col_filtering(
    dataframe=products,
    sim_matrix=item_sim,
    product_id=target_item,
    )

In [ ]:
from collections import Counter

def hybrid_recommend(dataframe, rec1, rec2, rec3, top_k=5, **alpha):
    
    weight1 = alpha.get('weight1')
    weight2 = alpha.get('weight2')
    weight3 = alpha.get('weight3')
    
    # 1. 추천된 상품들을 하나의 리스트로 합치기
    all_products = []
    
    all_products = rec1['product_id'].tolist()*weight1 \
        + rec2['product_id'].tolist()*weight2 \
        + rec3['product_id'].tolist()*weight3
        
    # 2. 상품별 등장 횟수 계산 (가중치 합산)
    product_counts = Counter(all_products)
    
    # 3. 등장 횟수 순으로 정렬하여 상위 top_k개 선택
    top_products = [prod_id for prod_id, count in product_counts.most_common(top_k)]

    # 4. 상품 정보 반환
    result = dataframe[dataframe['product_id'].isin(top_products)].copy()
    result['recommendation_score'] = result['product_id'].map(product_counts)
    
    return result[['product_id', 'name', 'category', 'brand', 'recommendation_score']].sort_values('recommendation_score', ascending=False)

print("🔹 하이브리드 추천")
alpha = dict(weight1=3, weight2=1, weight3=2)

hybrid_recommend(
    dataframe = products,
    rec1 = content_rec,
    rec2 = user_rec,
    rec3 = item_rec,
    **alpha
    )